In [562]:
#pip install pycoingecko
#pip install yahooquery
#import modules
import pandas as pd
import datetime
import time as t
import plotly.graph_objects as go
from plotly.offline import plot
from pycoingecko import CoinGeckoAPI
import yahooquery as yq
from yahooquery import Ticker
import glob
import os
import requests.exceptions
import responses
import datetime as dt
#create a client
cg = CoinGeckoAPI()

CRYPTO

In [563]:
#read and load the file path  
path=(r'/Users/sarahazad/Desktop/final_project/data collection/Resources')
all_files = glob.glob(os.path.join(path, "*.csv"))
#store in a dictonary
data={}
# itterate through paths, and read_csv and store it in dictonary
for i in all_files:
    data[i]=pd.read_csv(i)
#get dictonary values
BTC=data.get('/Users/sarahazad/Desktop/final_project/data collection/Resources/BTC.csv')
ETH=data.get('/Users/sarahazad/Desktop/final_project/data collection/Resources/ETH.csv')
XRP=data.get('/Users/sarahazad/Desktop/final_project/data collection/Resources/XRP.csv')
ADA=data.get('/Users/sarahazad/Desktop/final_project/data collection/Resources/ADA.csv')
# isinstance(BTC, pd.DataFrame)


In [564]:
#clean the historical crypto data
def clean(df):
#     df=pd.DataFrame(df) #not working
# dropping columns
    df.pop('unix')
    df.pop("symbol")
# dropping null values    
    df.dropna()
# renaming columns    
    df.rename(columns={'date':'Date','open':'Open','high':'High','low':'Low','close':'Close','Volume USD':'Volume'},inplace=True) 
# convert into datetime 
    df['Date']=pd.to_datetime(df['Date'])
    return df

# calling the function
BTC_df=clean(BTC)
ETH_df=clean(ETH)
XRP_df=clean(XRP)
ADA_df=clean(ADA)

# dropping column
BTC_df.pop('Volume BTC')
ETH_df.pop('Volume ETH')
ADA_df.pop('Volume ADA')
XRP_df.pop('Volume XRP')

# store it in dataframes
BTC_df=pd.DataFrame(BTC_df)
ETH_df=pd.DataFrame(ETH_df)
XRP=pd.DataFrame(XRP_df)
ADA_df=pd.DataFrame(ADA_df)



In [565]:
BTC_df.head()

,Date,Open,High,Low,Close,Volume
0,2022-10-26,20105.0,20170.0,20077.0,20143.0,5.776770e+05
1,2022-10-25,19339.0,20432.0,19251.0,20104.0,5.544651e+07
2,2022-10-24,19586.0,19615.0,19174.0,19340.0,2.537289e+07
3,2022-10-23,19219.0,19707.0,19086.0,19585.0,2.971904e+07
4,2022-10-22,19175.0,19278.0,19124.0,19219.0,1.447901e+07


In [566]:
#coingeko volume api call
coins = ['bitcoin','ethereum','ripple','cardano','solana','dodgecoin','polkadot']
marketcap_vol_change = cg.get_price(ids = coins, 
                        vs_currencies = 'usd', 
                        include_market_cap = True,
                        include_24hr_vol = True,
                        include_24hr_change = True,
                        include_last_updated_at = True)
#dataframe and transposed
# marketcap_vol_change_df=pd.DataFrame(marketcap_vol_change).T
# marketcap_vol_change_df.rename(columns={'usd':'price','usd_market_cap':'Market Cap','last_updated_at':'date','usd_24h_vol':'volume'},inplace=True) 

#coingeko OHLC API call

btc_data = cg.get_coin_ohlc_by_id(id = 'bitcoin', 
                                  vs_currency = 'usd', 
                                  days = '7')
eth_data = cg.get_coin_ohlc_by_id(id = 'ethereum', 
                                  vs_currency = 'usd', 
                                  days = '7')
xrp_data = cg.get_coin_ohlc_by_id(id = 'ripple', 
                                  vs_currency = 'usd', 
                                  days = '7')
ada_data = cg.get_coin_ohlc_by_id(id = 'cardano', 
                                  vs_currency = 'usd', 
                                  days = '7')

#dataframe

def dataframe(coin):
    columns = ['Date', 'Open', 'High' ,'Low', 'Close']
    coin=pd.DataFrame(coin,columns=columns)
    coin['Date'] = coin['Date'].apply(lambda x: dt.datetime.fromtimestamp(x/1000).strftime('%Y-%m-%d'))
    coin['Date']=pd.to_datetime(coin['Date'])
#     coin = coin.set_index('Date')
    coin=coin.sort_values(by=['Date'],ascending=False)
    coin=coin.iloc[[0]]
    return coin

# calling function
btc=dataframe(btc_data)
eth=dataframe(eth_data)
xrp=dataframe(xrp_data)
ada=dataframe(ada_data)

# isinstance(btc,pd.DataFrame)

In [567]:
# storing volume in OHLC dataframe
def volume():
    
    for x in coins:      
        if (x=='bitcoin'):
            btc['Volume']=marketcap_vol_change['bitcoin'].get('usd_24h_vol')
        elif (x=='ethereum'):
            eth['Volume']=marketcap_vol_change['ethereum'].get('usd_24h_vol')
        elif (x=='ripple'):
            xrp['Volume']=marketcap_vol_change['ripple'].get('usd_24h_vol')
        elif (x=='cardano'):
            ada['Volume']=marketcap_vol_change['cardano'].get('usd_24h_vol')
volume()

In [568]:
# adding row in in historical dataframe

BTC_df=pd.concat([btc,BTC_df])

ETH_df=pd.concat([eth,ETH_df])

ADA_df=pd.concat([ada,ADA_df])

XRP_df=pd.concat([xrp,XRP_df])


In [569]:
ETH_df.head()

,Date,Open,High,Low,Close,Volume
41,2022-10-28,1496.98,1505.06,1496.98,1502.69,1.748560e+10
0,2022-10-26,1461.10,1473.90,1458.50,1469.40,2.016825e+06
1,2022-10-25,1344.60,1533.10,1335.90,1460.80,6.977251e+07
2,2022-10-24,1365.30,1371.50,1324.70,1344.70,1.826617e+07
3,2022-10-23,1314.80,1372.40,1300.10,1364.80,1.536107e+07


In [572]:
# #reformat volume
# marketcap_vol_change_df['price'] = marketcap_vol_change_df['price'].map("{:,.2f}".format)
# marketcap_vol_change_df['volume'] = marketcap_vol_change_df['volume'].map("{:,.2f}".format)
# marketcap_vol_change_df['market_cap'] = marketcap_vol_change_df['market_cap'].map("{:,.2f}".format)
# marketcap_vol_change_df

STOCK

In [311]:
# Tesla Facebook Microsoft Apple Google
symbols = 'meta aapl msft goog tsla' 
# Make asynchronous requests
stock_data = Ticker(symbols, asynchronous=True)
stock_data

In [312]:
# View data available through the 'Summary' tab
summary_details = stock_data.summary_detail


In [81]:
#dataframe and transposed
stock_df=pd.DataFrame(summary_details).T
stock_df.columns


Index(['maxAge', 'priceHint', 'previousClose', 'open', 'dayLow', 'dayHigh',
       'regularMarketPreviousClose', 'regularMarketOpen',
       'regularMarketDayLow', 'regularMarketDayHigh', 'payoutRatio', 'beta',
       'trailingPE', 'forwardPE', 'volume', 'regularMarketVolume',
       'averageVolume', 'averageVolume10days', 'averageDailyVolume10Day',
       'bid', 'ask', 'bidSize', 'askSize', 'marketCap', 'fiftyTwoWeekLow',
       'fiftyTwoWeekHigh', 'priceToSalesTrailing12Months', 'fiftyDayAverage',
       'twoHundredDayAverage', 'trailingAnnualDividendRate',
       'trailingAnnualDividendYield', 'currency', 'fromCurrency', 'toCurrency',
       'lastMarket', 'coinMarketCapLink', 'algorithm', 'tradeable',
       'dividendRate', 'dividendYield', 'exDividendDate',
       'fiveYearAvgDividendYield'],
      dtype='object')

In [82]:
#Desired columns
columns=['open','dayLow', 'dayHigh',
       'regularMarketPreviousClose', 'regularMarketOpen',
       'regularMarketDayLow', 'regularMarketDayHigh','volume', 'regularMarketVolume']
stock_df = stock_df.loc[:, columns].copy()
stock_df

,open,dayLow,dayHigh,regularMarketPreviousClose,regularMarketOpen,regularMarketDayLow,regularMarketDayHigh,volume,regularMarketVolume
goog,98.46,98.23,101.62,100.53,98.46,98.23,101.62,27862280,27862280
aapl,142.87,142.649,147.8473,143.39,142.87,142.649,147.8473,86548609,86548609
meta,126.31,125.45,130.08,131.53,126.31,125.45,130.08,43918643,43918643
tsla,206.415,203.81,214.66,207.28,206.415,203.81,214.66,75713754,75713754
msft,234.74,234.5,242.88,236.15,234.74,234.5,242.88,25235911,25235911
